In [1]:
import pickle

In [2]:
rets = {}

In [5]:
path = "anymal_tanh_pd_ret"
with open("{}.pkl".format(path), "rb+") as file:
    data = pickle.load(file)

In [6]:
# resort
for seed, epi_data in data.items():
    for obs_index, obs_ret in epi_data.items():
        if obs_index == "seed":
            continue
        resort = sorted(obs_ret, key=lambda x: x["neuron"]["neuron_index"], reverse=True)
        resort = sorted(resort, key=lambda x: x["neuron"]["layer"], reverse=True)
        epi_data[obs_index] = resort


In [39]:
# average
averaged_ret = {}
for seed, epi_data in data.items():
    for obs, obs_ret in epi_data.items():
        if obs == "seed":
            continue
        if obs in averaged_ret:
            for k,each_neuron in enumerate(obs_ret):
                assert averaged_ret[obs][k]["layer"] == each_neuron["neuron"]["layer"] and averaged_ret[obs][k]["neuron_index"] == each_neuron["neuron"]["neuron_index"], "{} {}".format(averaged_ret[obs][k]["neuron_index"], each_neuron["neuron"]["neuron_index"])
                averaged_ret[obs][k]["freq_diff"] = (averaged_ret[obs][k]["freq_diff"]*averaged_ret[obs][k]["seed_num"] + each_neuron["error"]["freq_diff"])/(averaged_ret[obs][k]["seed_num"]+1)
                averaged_ret[obs][k]["base_phase_diff"] = (averaged_ret[obs][k]["base_phase_diff"]*averaged_ret[obs][k]["seed_num"] + each_neuron["error"]["base_phase_diff"])/(averaged_ret[obs][k]["seed_num"]+1)
                averaged_ret[obs][k]["seed_num"]+=1
        else:
            averaged_ret[obs] = []
            for each_neuron in obs_ret:
                averaged_ret[obs].append({"layer": each_neuron["neuron"]["layer"],
                                   "neuron_index": each_neuron["neuron"]["neuron_index"],
                                   "freq_diff": each_neuron["error"]["freq_diff"],
                                   "base_phase_diff": each_neuron["error"]["base_phase_diff"],
                                   "seed_num": 1})
for obs_dim, avg_ret in averaged_ret.items():
    averaged_ret[obs_dim] = sorted(avg_ret, key=lambda x:x["freq_diff"],reverse=False)

rets[path] = averaged_ret

In [40]:
for key, value in averaged_ret.items():
    remove = []
    for k,unit in enumerate(value):
        if unit["neuron_index"]>=128 and unit["layer"]==0:
            remove.append(unit)
        elif unit["neuron_index"]>=64 and unit["layer"]==1:
            remove.append(unit)
        elif unit["neuron_index"]>=32 and unit["layer"]==2:
            remove.append(unit)
    for k in remove:
        value.remove(k)

In [41]:
print(averaged_ret[6])

[{'layer': 0, 'neuron_index': 121, 'freq_diff': 42.424372044172955, 'base_phase_diff': 0.9194762292298229, 'seed_num': 10}, {'layer': 0, 'neuron_index': 119, 'freq_diff': 44.193003549604214, 'base_phase_diff': 0.7538160000470518, 'seed_num': 10}, {'layer': 0, 'neuron_index': 24, 'freq_diff': 44.216828564992305, 'base_phase_diff': 0.9064437031571565, 'seed_num': 10}, {'layer': 0, 'neuron_index': 20, 'freq_diff': 44.6024284002798, 'base_phase_diff': 0.9058913562128094, 'seed_num': 10}, {'layer': 0, 'neuron_index': 65, 'freq_diff': 44.64473199545344, 'base_phase_diff': 0.9326427464730616, 'seed_num': 10}, {'layer': 0, 'neuron_index': 112, 'freq_diff': 46.183562742259966, 'base_phase_diff': 0.9549892608330974, 'seed_num': 10}, {'layer': 0, 'neuron_index': 31, 'freq_diff': 47.70947704615893, 'base_phase_diff': 0.9298973862322868, 'seed_num': 10}, {'layer': 0, 'neuron_index': 97, 'freq_diff': 47.984588754759706, 'base_phase_diff': 0.9308416269489721, 'seed_num': 10}, {'layer': 0, 'neuron_ind

In [ ]:

# 12-23  +1
# 24-35 +1 497 230 40
print(rets["cassie_0_z_pd_ret"][0])